<div style="background-color: #04D7FD; padding: 20px; text-align: left;">
    <h1 style="color: #000000; font-size: 36px; margin: 0;">Data Processing for RAG with Data Prep Kit (RAY)</h1>
    
</div>


## Before Running the notebook

Please complete [setting up python dev environment](./setup-python-dev-env.md)

## Overview

This notebook will process PDF documents as part of RAG pipeline

![](media/rag-overview-2.png)

This notebook will perform steps 1, 2 and 3 in RAG pipeline.

Here are the processing steps:

- **pdf2parquet** : Extract text from PDF and convert them into parquet files
- **Chunk documents**: Split the PDFs into 'meaningful sections' (paragraphs, sentences ..etc)
- **Doc_ID generation**: Each chunk is assigned a uniq id, based on content and hash
- **Exact Dedup**: Chunks with exact same content are filtered out
- **Fuzzy Dedup**: Eliminate chunks that are 'very similar' content
- **Doc quality**: Scores the documents based on criteria like number of words, if it contains bad words ..etc
- **Text encoder**: Convert chunks into vectors using embedding models

## Step-1: Configuration

### 1.1 - Basic Config

In [1]:
import os 

## Configuration
class MyConfig:
    pass 

MY_CONFIG = MyConfig ()

## Input Data - configure this to the folder we want to process
MY_CONFIG.INPUT_DATA_DIR = "../data/papers/"
MY_CONFIG.OUTPUT_FOLDER = "output"
MY_CONFIG.OUTPUT_FOLDER_FINAL = os.path.join(MY_CONFIG.OUTPUT_FOLDER , "output_final")

## Embedding model
MY_CONFIG.EMBEDDING_MODEL = 'sentence-transformers/all-MiniLM-L6-v2'

## RAY CONFIGURATION
num_cpus_available =  os.cpu_count()
# print (num_cpus_available)
# MY_CONFIG.RAY_NUM_CPUS = num_cpus_available // 2  ## use half the available cores for processing
MY_CONFIG.RAY_NUM_CPUS =  1
# print (MY_CONFIG.RAY_NUM_CPUS)
MY_CONFIG.RAY_MEMORY_GB = 2  # GB
# MY_CONFIG.RAY_RUNTIME_WORKERS = num_cpus_available // 3
MY_CONFIG.RAY_RUNTIME_WORKERS = 2

### 1.2 - Inspect Input Data

We have a bnunch of datasets in [data](../data) folder.  Examine them

We will use one of them or feel free to bring your own data.

### 1.3  - Set input/output path variables for the pipeline

In [2]:
import os, sys
import shutil

if not os.path.exists(MY_CONFIG.INPUT_DATA_DIR ):
    raise Exception (f"❌ Input folder MY_CONFIG.INPUT_DATA_DIR = '{MY_CONFIG.INPUT_DATA_DIR}' not found")

output_parquet_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '01_parquet_out')
output_chunk_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '02_chunk_out')
output_docid_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '03_docid_out')
output_exact_dedupe_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '04_exact_dedupe_out')
output_fuzzy_dedupe_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '05_fuzzy_dedupe_out')
output_embeddings_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '06_embeddings_out')


## clear output folder
shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER, ignore_errors=True)
shutil.os.makedirs(MY_CONFIG.OUTPUT_FOLDER, exist_ok=True)

print ("✅ Cleared output directory")

✅ Cleared output directory


### 1.4 - Import Common python modules

In [3]:
import os
import sys


from data_processing_ray.runtime.ray import RayTransformLauncher
from data_processing.utils import ParamsUtils

<a id="pdf2parquet"></a>

## Step-2: pdf2parquet -  Convert data from PDF to Parquet

This step is reading the input folder containing all PDF files and ingest them in a parquet table using the [Docling package](https://github.com/DS4SD/docling).
The documents are converted into a JSON format which allows to easily chunk it in the later steps.



### 2.1 - Set Input/output Folder

In [4]:
STAGE = 1 

input_folder = MY_CONFIG.INPUT_DATA_DIR
output_folder =  output_parquet_dir

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-1: Processing input='../data/papers/' --> output='output/01_parquet_out'


### 2.2 -  Execute 

In [5]:
%%time 

import ast
import os
import sys

from pdf2parquet_transform import (
    pdf2parquet_contents_type_cli_param,
    pdf2parquet_contents_types,
)
from pdf2parquet_transform_python import Pdf2ParquetPythonTransformConfiguration
from pdf2parquet_transform_ray import Pdf2ParquetRayTransformConfiguration

from data_processing.utils import GB, ParamsUtils


# create parameters
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS, "memory": MY_CONFIG.RAY_MEMORY_GB * GB}
code_location = {"github": "github", "commit_hash": "12345", "path": "path"}
ingest_config = {
    pdf2parquet_contents_type_cli_param: pdf2parquet_contents_types.JSON,
}

params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    "data_files_to_use": ast.literal_eval("['.pdf']"),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    "runtime_pipeline_id": "pipeline_id",
    "runtime_job_id": "job_id",
    "runtime_code_location": ParamsUtils.convert_to_ast(code_location),
}


sys.argv = ParamsUtils.dict_to_req(d=(params | ingest_config))
# create launcher
launcher = RayTransformLauncher(Pdf2ParquetRayTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")


22:08:47 INFO - Running locally
22:08:47 INFO - pdf2parquet parameters are : {'artifacts_path': None, 'contents_type': <pdf2parquet_contents_types.JSON: 'application/json'>, 'do_table_structure': True, 'do_ocr': True, 'double_precision': 8}
22:08:47 INFO - data factory data_ is using local data access: input_folder - ../data/papers/ output_folder - output/01_parquet_out
22:08:47 INFO - data factory data_ max_files -1, n_sample -1
22:08:47 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.pdf'], files to checkpoint ['.parquet']
22:08:47 INFO - pipeline id pipeline_id
22:08:47 INFO - code location {'github': 'github', 'commit_hash': '12345', 'path': 'path'}
22:08:47 INFO - number of workers 2 worker options {'num_cpus': 1, 'memory': 2147483648, 'max_restarts': -1}
22:08:47 INFO - actor creation delay 0
22:08:47 INFO - job details {'job category': 'preprocessing', 'job name': 'pdf2parquet', 'job type': 'ray', 'job id': 'jo

✅ Stage:1 completed successfully
CPU times: user 3.74 s, sys: 517 ms, total: 4.25 s
Wall time: 2min 23s


### 2.3 - Inspect Generated output

Here we should see one entry per input file processed

In [6]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Output dimensions (rows x columns)= ", output_df.shape)

output_df.head(5)

## To display certain columns
#parquet_df[['column1', 'column2', 'column3']].head(5)

Output dimensions (rows x columns)=  (2, 12)


,filename,contents,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename
0,attention-is-all-you-need.pdf,"{""_name"":"""",""type"":""pdf-document"",""description...",15,4,193,1dae5518-732a-4963-944d-e2f549b08b51,pdf,414d95bff49753a945e0917d47b840bef75c67bd56af13...,137574,2024-09-18T22:09:54.177118,54.783175,attention-is-all-you-need.pdf
1,Granite_Foundation_Models.pdf,"{""_name"":"""",""type"":""pdf-document"",""description...",20,13,444,5deee7bb-f13b-42eb-b8b5-c65cfad5dd40,pdf,f64d5f0f36008888c7862a0ec33a06255f3e71f85f02ca...,394452,2024-09-18T22:10:57.604339,118.290917,Granite_Foundation_Models.pdf


<a id="chunking"></a>

##  Step-3: Doc chunks

Split the documents in chunks, according to their layout segmentation.

### 3.1 - Set Input/output Folder

In [7]:
STAGE = 2

input_folder = output_parquet_dir # previous output folder is the input folder for the current stage
output_folder =  output_chunk_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-2: Processing input='output/01_parquet_out' --> output='output/02_chunk_out'


### 3.2 -  Execute 

In [8]:
%%time 

# Import doc_json_chunk transform configuration
from doc_chunk_transform_ray import DocChunkRayTransformConfiguration


# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # doc_chunk arguments
    # ...
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = RayTransformLauncher(DocChunkRayTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

22:11:10 INFO - Running locally
22:11:10 INFO - doc_chunk parameters are : {'chunking_type': <chunking_types.DL_JSON: 'dl_json'>, 'content_column_name': 'contents', 'output_chunk_column_name': 'contents', 'output_jsonpath_column_name': 'doc_jsonpath', 'output_pageno_column_name': 'page_number', 'output_bbox_column_name': 'bbox'}
22:11:10 INFO - data factory data_ is using local data access: input_folder - output/01_parquet_out output_folder - output/02_chunk_out
22:11:10 INFO - data factory data_ max_files -1, n_sample -1
22:11:10 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
22:11:10 INFO - pipeline id pipeline_id
22:11:10 INFO - code location None
22:11:10 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
22:11:10 INFO - actor creation delay 0
22:11:10 INFO - job details {'job category': 'preprocessing', 'job name': 'doc_chunk', 'job type': 'r

✅ Stage:2 completed successfully
CPU times: user 974 ms, sys: 303 ms, total: 1.28 s
Wall time: 19.2 s


### 3.3 - Inspect Generated output

We would see documents are split into many chunks

In [9]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print (f"Files processed : {input_df.shape[0]:,}")
print (f"Chunks created : {output_df.shape[0]:,}")

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Files processed : 2
Chunks created : 266
Input data dimensions (rows x columns)=  (2, 12)
Output data dimensions (rows x columns)=  (266, 15)


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox
25,attention-is-all-you-need.pdf,15,4,193,1dae5518-732a-4963-944d-e2f549b08b51,pdf,414d95bff49753a945e0917d47b840bef75c67bd56af13...,137574,2024-09-18T22:09:54.177118,54.783175,attention-is-all-you-need.pdf,3.2.1 Scaled Dot-Product Attention\nWe call ou...,$.main-text[45],4,"[107.08164215, 379.78808594, 504.3180542, 422...."
39,attention-is-all-you-need.pdf,15,4,193,1dae5518-732a-4963-944d-e2f549b08b51,pdf,414d95bff49753a945e0917d47b840bef75c67bd56af13...,137574,2024-09-18T22:09:54.177118,54.783175,attention-is-all-you-need.pdf,"3.4 Embeddings and Softmax\nSelf-Attention, Co...",$.tables[0],6,"[117.776474, 605.01715088, 493.66720581, 679.6..."
175,Granite_Foundation_Models.pdf,20,13,444,5deee7bb-f13b-42eb-b8b5-c65cfad5dd40,pdf,f64d5f0f36008888c7862a0ec33a06255f3e71f85f02ca...,394452,2024-09-18T22:10:57.604339,118.290917,Granite_Foundation_Models.pdf,B. Granite Model Evaluation and Comparison\npr...,$.main-text[184],8,"[311.06741333, 132.49633789, 563.21166992, 154..."


## Step-4:  DOC ID generation

This transform annotates documents with document "ids". It supports the following transformations of the original data:

 - Adding document hash: this enables the addition of a document hash-based id to the data. The hash is calculated with `hashlib.sha256(doc.encode("utf-8")).hexdigest()`. To enable this annotation, set hash_column to the name of the column, where you want to store it.
 - Adding integer document id: this allows the addition of an integer document id to the data that is unique across all rows in all tables provided to the transform() method. To enable this annotation, set int_id_column to the name of the column, where you want to store it. **This is a pre-requisite for fuzzy dedup** in the pipeline.

### 4.1 - Set Input/output Folder

In [10]:

STAGE  = 3

input_folder = output_chunk_dir # previous output folder is the input folder for the current stage
output_folder =  output_docid_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-3: Processing input='output/02_chunk_out' --> output='output/03_docid_out'


### 4.2 - Execute 

In [11]:
%%time 

from doc_id_transform_ray import DocIDRayTransformRuntimeConfiguration
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # doc id configuration
    "doc_id_doc_column": "contents",
    "doc_id_hash_column": "chunk_hash",
    "doc_id_int_column": "chunk_id",
}
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = RayTransformLauncher(DocIDRayTransformRuntimeConfiguration())

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

22:11:28 INFO - Running locally
22:11:28 INFO - Doc id parameters are : {'doc_column': 'contents', 'hash_column': 'chunk_hash', 'int_column': 'chunk_id', 'start_id': 0}
22:11:28 INFO - data factory data_ is using local data access: input_folder - output/02_chunk_out output_folder - output/03_docid_out
22:11:28 INFO - data factory data_ max_files -1, n_sample -1
22:11:28 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
22:11:28 INFO - pipeline id pipeline_id
22:11:28 INFO - code location None
22:11:28 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
22:11:28 INFO - actor creation delay 0
22:11:28 INFO - job details {'job category': 'preprocessing', 'job name': 'doc_id', 'job type': 'ray', 'job id': 'job_id'}
2024-09-18 22:11:30,169	INFO worker.py:1744 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
(orchestrate pid=143

✅ Stage:3 completed successfully
CPU times: user 115 ms, sys: 147 ms, total: 262 ms
Wall time: 15.2 s


### 4.3 - Inspect Generated output

In [12]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (266, 15)
Output data dimensions (rows x columns)=  (266, 17)


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox,chunk_hash,chunk_id
10,attention-is-all-you-need.pdf,15,4,193,1dae5518-732a-4963-944d-e2f549b08b51,pdf,414d95bff49753a945e0917d47b840bef75c67bd56af13...,137574,2024-09-18T22:09:54.177118,54.783175,attention-is-all-you-need.pdf,1 Introduction\nRecurrent models typically fac...,$.main-text[19],2,"[107.33866882, 546.35638428, 504.39059448, 632...",dea125ef05786922230bf502fbec5fd9332783c2eb3daa...,10
259,Granite_Foundation_Models.pdf,20,13,444,5deee7bb-f13b-42eb-b8b5-c65cfad5dd40,pdf,f64d5f0f36008888c7862a0ec33a06255f3e71f85f02ca...,394452,2024-09-18T22:10:57.604339,118.290917,Granite_Foundation_Models.pdf,JAPANESE FINANCE BENCHMARK EVALUATION RESULTS\...,$.tables[9],18,"[66.48503876, 285.29275513, 545.31109619, 335....",6473880114c955f6202666fd9e3aa4e6a6fab6582ed56b...,259
152,Granite_Foundation_Models.pdf,20,13,444,5deee7bb-f13b-42eb-b8b5-c65cfad5dd40,pdf,f64d5f0f36008888c7862a0ec33a06255f3e71f85f02ca...,394452,2024-09-18T22:10:57.604339,118.290917,Granite_Foundation_Models.pdf,A. Foundation Model Evaluation Framework\nThes...,$.main-text[147],6,"[310.76846313, 286.52426147, 563.35235596, 380...",c19003ce8cdf42e8b0d6cdf7e2bd911c76a17085436024...,152


## Step-5: Exact Dedup

Remove documents having identical code to remove bias in the training data. On the content of each document, a SHA256 hash is computed,
followed by de-duplication of record having identical hashes.

### 5.1 -  Set Input/output Folder

In [13]:
STAGE  = 4

input_folder = output_docid_dir # previous output folder is the input folder for the current stage
output_folder =  output_exact_dedupe_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-4: Processing input='output/03_docid_out' --> output='output/04_exact_dedupe_out'


### 5.2 - Execute 

In [14]:
%%time

# Import ededup transform configuration
from ededup_transform_ray import EdedupRayTransformRuntimeConfiguration


# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # ededup parameters
    "ededup_hash_cpu": 0.5,
    "ededup_num_hashes": 2,
    "ededup_doc_column": "contents",
    "ededup_doc_id_column": "chunk_hash",
    
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = RayTransformLauncher(EdedupRayTransformRuntimeConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

22:11:43 INFO - Running locally
22:11:43 INFO - exact dedup params are {'doc_column': 'contents', 'doc_id_column': 'chunk_hash', 'use_snapshot': False, 'snapshot_directory': None, 'hash_cpu': 0.5, 'num_hashes': 2}
22:11:43 INFO - data factory data_ is using local data access: input_folder - output/03_docid_out output_folder - output/04_exact_dedupe_out
22:11:43 INFO - data factory data_ max_files -1, n_sample -1
22:11:43 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
22:11:43 INFO - pipeline id pipeline_id
22:11:43 INFO - code location None
22:11:43 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
22:11:43 INFO - actor creation delay 0
22:11:43 INFO - job details {'job category': 'preprocessing', 'job name': 'ededup', 'job type': 'ray', 'job id': 'job_id'}
2024-09-18 22:11:45,443	INFO worker.py:1744 -- Started a local Ray instance. View the dash

✅ Stage:4 completed successfully
CPU times: user 127 ms, sys: 161 ms, total: 288 ms
Wall time: 15 s


### 5.3 - Inspect Generated output

In [15]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print (f"Input chunks before exact dedupe : {input_df.shape[0]:,}")
print (f"Output chunks after exact dedupe : {output_df.shape[0]:,}")
print ("Duplicate chunks removed :  ", (input_df.shape[0] - output_df.shape[0]))

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (266, 17)
Output data dimensions (rows x columns)=  (266, 18)
Input chunks before exact dedupe : 266
Output chunks after exact dedupe : 266
Duplicate chunks removed :   0


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox,chunk_hash,chunk_id,removed
139,Granite_Foundation_Models.pdf,20,13,444,5deee7bb-f13b-42eb-b8b5-c65cfad5dd40,pdf,f64d5f0f36008888c7862a0ec33a06255f3e71f85f02ca...,394452,2024-09-18T22:10:57.604339,118.290917,Granite_Foundation_Models.pdf,A. Algorithmic Details\nFig. 4. An (a) archite...,$.main-text[121],5,"[311.10406494, 399.15835571, 537.22070312, 416...",e7fd34ae9e9e920ee469536aacd12946294557b1d51ea7...,139,[]
23,attention-is-all-you-need.pdf,15,4,193,1dae5518-732a-4963-944d-e2f549b08b51,pdf,414d95bff49753a945e0917d47b840bef75c67bd56af13...,137574,2024-09-18T22:09:54.177118,54.783175,attention-is-all-you-need.pdf,3.2 Attention\nFigure 2: (left) Scaled Dot-Pro...,$.main-text[41],4,"[107.44569397, 496.87033081, 503.99707031, 517...",57b10273e28c8be40591125396fd35fa1fef07341e0717...,23,[]
39,attention-is-all-you-need.pdf,15,4,193,1dae5518-732a-4963-944d-e2f549b08b51,pdf,414d95bff49753a945e0917d47b840bef75c67bd56af13...,137574,2024-09-18T22:09:54.177118,54.783175,attention-is-all-you-need.pdf,"3.4 Embeddings and Softmax\nSelf-Attention, Co...",$.tables[0],6,"[117.776474, 605.01715088, 493.66720581, 679.6...",82bb3caaa851315dd141eeb2d7b9cb7b2893b2378510b9...,39,[]


## Step-6: Fuzzy Dedup

Post exact deduplication, fuzzy deduplication is applied with
the goal of removing code files that may have slight variations and thereby unbiasing
the data further. Small variations are quite commonly seen in code data in the form
of variations in the values of variables, addittion of logging statements etc. Find near-
duplicate.

### 6.1 - Set Input/output Folder

In [16]:
## Input to this component is the output of doc_id generator component. 

STAGE  = 5

input_folder = output_exact_dedupe_dir # previous output folder is the input folder for the current stage
output_folder =  output_fuzzy_dedupe_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-5: Processing input='output/04_exact_dedupe_out' --> output='output/05_fuzzy_dedupe_out'


### 6.2 - Execute 

In [17]:
%%time 

import os
import sys

from data_processing.utils import ParamsUtils
from fdedup_transform_ray import FdedupRayTransformConfiguration

# create parameters

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
code_location = {"github": "github", "commit_hash": "12345", "path": "path"}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # Orchestration parameters
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # columns used
    "fdedup_doc_column": "contents",
    "fdedup_id_column": "chunk_id",
    "fdedup_cluster_column": "chunk_hash",
    # infrastructure
    "fdedup_bucket_cpu": 0.5,
    "fdedup_doc_cpu": 0.5,
    "fdedup_mhash_cpu": 0.5,
    "fdedup_num_doc_actors": 2,
    "fdedup_num_bucket_actors": 1,
    "fdedup_num_minhash_actors": 1,
    "fdedup_num_preprocessors": 2,
    # fuzzy parameters
    "fdedup_num_permutations": 64,
    "fdedup_threshold": 0.7, # between 0.0 to 1.0 ; smaller values tend to be more lenient in finding near dupes; close to 1.0 is more strict
    "fdedup_shingles_size": 5,
    "fdedup_delimiters": " "
}

# Pass commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = RayTransformLauncher(FdedupRayTransformConfiguration())

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

22:11:58 INFO - Running locally
22:11:58 INFO - fuzzy dedup params are {'doc_column': 'contents', 'id_column': 'chunk_id', 'cluster_column': 'chunk_hash', 'bucket_cpu': 0.5, 'mhash_cpu': 0.5, 'doc_cpu': 0.5, 'num_doc_actors': 2, 'num_minhash_actors': 1, 'num_bucket_actors': 1, 'num_preprocessors': 2, 'num_permutations': 64, 'threshold': 0.7, 'shingles_size': 5, 'delimiters': ' ', 'snapshot_delay': 1, 'use_bucket_snapshot': False, 'use_doc_snapshot': False, 'random_delay_limit': 10, 'worker_options': {'num_cpus': 1}}
22:11:58 INFO - data factory data_ is using local data access: input_folder - output/04_exact_dedupe_out output_folder - output/05_fuzzy_dedupe_out
22:11:58 INFO - data factory data_ max_files -1, n_sample -1
22:11:58 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
22:11:58 INFO - pipeline id pipeline_id
22:11:58 INFO - code location None
22:11:58 INFO - number o

✅ Stage:5 completed successfully
CPU times: user 192 ms, sys: 201 ms, total: 393 ms
Wall time: 35 s


### 6.3 - Inspect Generated output

In [18]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print ("Duplicate chunks removed  by fuzzy-dedupe:  ", (input_df.shape[0] - output_df.shape[0]))

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (266, 18)
Output data dimensions (rows x columns)=  (266, 18)
Duplicate chunks removed  by fuzzy-dedupe:   0


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox,chunk_id,removed,chunk_hash
97,Granite_Foundation_Models.pdf,20,13,444,5deee7bb-f13b-42eb-b8b5-c65cfad5dd40,pdf,f64d5f0f36008888c7862a0ec33a06255f3e71f85f02ca...,394452,2024-09-18T22:10:57.604339,118.290917,Granite_Foundation_Models.pdf,B. Overview of the Granite Pre-Training Datase...,$.main-text[15],1,"[310.96148682, 84.31424713, 563.50097656, 177....",97,[],-1
246,Granite_Foundation_Models.pdf,20,13,444,5deee7bb-f13b-42eb-b8b5-c65cfad5dd40,pdf,f64d5f0f36008888c7862a0ec33a06255f3e71f85f02ca...,394452,2024-09-18T22:10:57.604339,118.290917,Granite_Foundation_Models.pdf,C. Evaluation\nThe model is evaluated with bot...,$.main-text[408],17,"[47.82268524, 96.21683502, 300.02154541, 118.3...",246,[],-1
17,attention-is-all-you-need.pdf,15,4,193,1dae5518-732a-4963-944d-e2f549b08b51,pdf,414d95bff49753a945e0917d47b840bef75c67bd56af13...,137574,2024-09-18T22:09:54.177118,54.783175,attention-is-all-you-need.pdf,3 Model Architecture\nMost competitive neural ...,$.main-text[28],2,"[107.17133331, 69.61254883, 505.74313354, 122....",17,[],-1


## Step-7:   Text encoding

Encode text for the vector storage.

### 7.1 - Set Input/output Folder

In [19]:
STAGE  = 6

input_folder = output_fuzzy_dedupe_dir
output_folder =  output_embeddings_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-6: Processing input='output/05_fuzzy_dedupe_out' --> output='output/06_embeddings_out'


### 7.2 - Execute

In [20]:
%%time 

from text_encoder_transform_ray import TextEncoderRayTransformConfiguration

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # text_encoder
    "text_encoder_model_name": MY_CONFIG.EMBEDDING_MODEL,
}

sys.argv = ParamsUtils.dict_to_req(d=params)
# create launcher
launcher = RayTransformLauncher(TextEncoderRayTransformConfiguration())
# Launch the ray actor(s) to process the input

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

22:12:34 INFO - Running locally
22:12:34 INFO - text_encoder parameters are : {'content_column_name': 'contents', 'output_embeddings_column_name': 'embeddings', 'model_name': 'sentence-transformers/all-MiniLM-L6-v2'}
22:12:34 INFO - data factory data_ is using local data access: input_folder - output/05_fuzzy_dedupe_out output_folder - output/06_embeddings_out
22:12:34 INFO - data factory data_ max_files -1, n_sample -1
22:12:34 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
22:12:34 INFO - pipeline id pipeline_id
22:12:34 INFO - code location None
22:12:34 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
22:12:34 INFO - actor creation delay 0
22:12:34 INFO - job details {'job category': 'preprocessing', 'job name': 'text_encoder', 'job type': 'ray', 'job id': 'job_id'}
2024-09-18 22:12:36,041	INFO worker.py:1744 -- Started a local Ray instance.

✅ Stage:6 completed successfully
CPU times: user 464 ms, sys: 228 ms, total: 691 ms
Wall time: 27.9 s


### 7.3 - Inspect Generated output

In [21]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (266, 18)
Output data dimensions (rows x columns)=  (266, 19)


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox,chunk_id,removed,chunk_hash,embeddings
258,Granite_Foundation_Models.pdf,20,13,444,5deee7bb-f13b-42eb-b8b5-c65cfad5dd40,pdf,f64d5f0f36008888c7862a0ec33a06255f3e71f85f02ca...,394452,2024-09-18T22:10:57.604339,118.290917,Granite_Foundation_Models.pdf,JAPANESE FINANCE BENCHMARK EVALUATION RESULTS\...,$.main-text[432],18,"[213.05499268, 244.48545837, 398.37347412, 261...",258,[],-1,"[-0.03375955, -0.034483366, -0.070505105, 0.06..."
22,attention-is-all-you-need.pdf,15,4,193,1dae5518-732a-4963-944d-e2f549b08b51,pdf,414d95bff49753a945e0917d47b840bef75c67bd56af13...,137574,2024-09-18T22:09:54.177118,54.783175,attention-is-all-you-need.pdf,3.2 Attention\nAn attention function can be de...,$.main-text[37],3,"[106.82707977, 69.26216125, 505.24853516, 89.9...",22,[],-1,"[0.047590945, -0.044376105, -0.060143728, 0.00..."
110,Granite_Foundation_Models.pdf,20,13,444,5deee7bb-f13b-42eb-b8b5-c65cfad5dd40,pdf,f64d5f0f36008888c7862a0ec33a06255f3e71f85f02ca...,394452,2024-09-18T22:10:57.604339,118.290917,Granite_Foundation_Models.pdf,III. DATA GOVERNANCE\nData governance is organ...,$.main-text[53],3,"[48.17564774, 322.20968628, 300.02474976, 344....",110,[],-1,"[0.022724738, -0.037658304, -0.036405515, -0.0..."


## Step-8: Copy output to final output dir

In [22]:
import shutil

shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER_FINAL, ignore_errors=True)
shutil.copytree(src=output_folder, dst=MY_CONFIG.OUTPUT_FOLDER_FINAL)

print (f"✅ Copied output from '{output_folder}' --> '{MY_CONFIG.OUTPUT_FOLDER_FINAL}'")

✅ Copied output from 'output/06_embeddings_out' --> 'output/output_final'
